### StyleGAN2-ada-pytorch 모델 사용

* git clone https://github.com/NVlabs/stylegan2-ada-pytorch
* 터미널에서 명령어 실행
* 필요 라이브러리 설치 pip install click requests tqdm pyspng ninja imageio-ffmpeg==0.4.3

In [2]:
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch

Cloning into 'stylegan2-ada-pytorch'...


In [6]:
!conda install click requests tqdm pyspng ninja imageio-ffmpeg==0.4.3

Channels:
 - defaults
 - conda-forge
 - anaconda
Platform: win-64
Solving environment: ...working... failed


C:\Users\user\MiniConda3\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'conda.anaconda.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\user\MiniConda3\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'conda.anaconda.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(

PackagesNotFoundError: The following packages are not available from current channels:

  - pyspng

Current channels:

  - defaults
  - https://conda.anaconda.org/conda-forge/noarch
  - https://conda.anaconda.org/conda-forge/win-64
  - https://conda.anaconda.org/anaconda/win-64

To search for alternate channels that may provide the conda package you're
looking fo

In [1]:
# stylegan2-ada-pytorch 폴더로 이동
!cd 'D:\project\GAN\3. styleGAN2-ada-pytorch\stylegan2-ada-pytorch'

���� �̸�, ���͸� �̸� �Ǵ� ���� ���̺� ������ �߸��Ǿ����ϴ�.


In [2]:
import os
os.chdir(r'D:\project\GAN\3. styleGAN2-ada-pytorch\stylegan2-ada-pytorch')
current = os.getcwd()
print(current)

D:\project\GAN\3. styleGAN2-ada-pytorch\stylegan2-ada-pytorch


### StyleGAN2-ada-pytorch는 torch 1.8버전에서 제일 호환성이 좋다고 한다.
* torch 1.8 버전과 그에 해당하는 cuda 설치
* conda install torch==1.8.0+cu111 torchvision==0.9.0+cu111 torchaudio==0.8.0 -f https://download.pytorch.org/whl/torch_stable.html
* torch 1.8 버전과 호환되는 numpy도 재설치
* conda install numpy=1.20

In [7]:
import torch
import numpy as np
print(torch.__version__)
print(np.__version__)

1.8.0+cu111
1.24.4


### 데이터셋 준비
* python dataset.py --source=./images --dest=./data/stylegan2-dataset.zip --width=512 --height=512


    * 1. python dataset_tool.py: dataset_tool.py 스크립트를 실행해서 원본 이미지들을 StyleGAN2 학습에 적합한 형식으로 변환
    * 2. --source=./images: 원본 이미지가 있는 디렉토리를 지정. 여기서는 현재 디렉토리의 'images' 폴더
    * 3. --dest=./data/stylegan2-dataset.zip: 변환된 데이터셋이 저장될 위치와 파일명 지정. 이 경우 'data' 폴더 안에 'stylegan2-dataset.zip'이라는 이름으로 저장
    * 4. --width=256 --height=256: 출력 이미지의 크기 설정

In [8]:
!python dataset_tool.py --source=./images --dest=./data/stylegan2-dataset.zip --width=512 --height=512


100%|██████████| 877/877 [00:20<00:00, 42.86it/s]


### 모델 학습
* python train.py --outdir=runs --data=./data/stylegan2-dataset.zip --gpus=1 --aug=ada --target=0.7
    * 1. --outdir=runs : 훈련 결과물이 저장될 디렉토리를 지정. 여기서는 'runs' 폴더에 저장
    * 2. --data=./data/stylegan2-dataset.zip : 훈련에 사용될 데이터셋의 위치를 지정. './data/stylegan2-dataset.zip' 파일에 데이터셋이 있음
    * 3. --gpus=1 : 훈련에 사용할 GPU의 수를 지정. 여기서는 1개의 GPU 사용
    * 4. --aug=ada : Adaptive Discriminator Augmentation (ADA) 기법을 사용하도록 설정
    * 5. --target=0.7 : ADA의 목표 값을 설정. 0.7로 설정하면 과도한 증강을 방지하면서 동적 증강을 최대화

* ffhq 데이터셋으로 pre-trained 된 pkl 파일을 사용해 전이학습(512*512)
    * https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/transfer-learning-source-nets/ffhq-res512-mirror-stylegan2-noaug.pkl

In [9]:
!python train.py --outdir=./runs --data=./data/stylegan2-dataset.zip --gpus=1 --kimg=1000 --aug=ada --target=0.7 --resume="D:\project\GAN\3. styleGAN2-ada-pytorch\ffhq_512.pkl"

^C


### 훈련 시간이 너무 오래 걸려서 데이터셋 리사이징
* 256*256으로 시도

In [3]:
!python dataset_tool.py --source=./images --dest=./data/dataset_256.zip --width=256 --height=256


100%|██████████| 877/877 [00:19<00:00, 46.02it/s]


### .pkl 파일의 저장 주기 설정
* 기본값은 50 kimg 주기로 저장하는데, 훈련 시간을 고려 5kimg로 수정
* --snap=5
* batch 사이즈도 16으로 수정
* --batch=16

In [ ]:
# ffhq로 사전훈련된 pkl 파일도 256*256으로 훈련된 모델로 바꿔줘야 한다
# https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/transfer-learning-source-nets/ffhq-res256-mirror-paper256-noaug.pkl
!python train.py --outdir=./runs --data=./data/dataset_256.zip --gpus=1 --kimg=1000 --batch=16 --aug=ada --target=0.7 --resume="D:\project\GAN\3. styleGAN2-ada-pytorch\ffhq_256.pkl" --snap=5

### docker를 사용한 trainig
* docker 설치 후 docker를 실행한 뒤에 진행
* docker build -t stylegan2ada .
    * docker build : Docker 이미지 생성
    * -t stylegan2ada : 생성될 이미지에 태그 지정. stylegan2ada 라는 이름의 태그 지정
    * . : 현재 디렉토리에 있는 docker 파일을 사용해 이미지 빌드

In [1]:
!docker build -t stylegan2ada .

#0 building with "desktop-linux" instance using docker driver

#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 2B done
#1 DONE 0.0s
ERROR: failed to solve: failed to read dockerfile: open Dockerfile: no such file or directory


In [5]:
!docker images

REPOSITORY     TAG       IMAGE ID       CREATED          SIZE
stylegan2ada   latest    b9b4cfc3db7d   20 minutes ago   21.1GB


### docker 내에서 데이터셋 training

* 로컬 경로를 docker와 마운트
    * 터미널에서 docker run --gpus all -it --rm -v ./data:/dataset -v ./runs:/results stylegan2ada /bin/bash
    * docker run : 새로운 docker 컨테이너 실행
    * --gpus all : 모든 gpu 사용
    * -it : 대화형 모드로 실행
    * --rm : 컨테이너가 종료될 때 자동으로 삭제
    * -v ./data:/dataset : 로컬의 ./data 디렉토리를 컨테이너의 /dataset 디렉토리에 마운트
    * -v ./runs:/results : 로컬의 ./runs 디렉토리를 컨테이너의 /results 디렉토리에 마운트
    * stylegan2ada : docker 이미지의 이름
    * /bin/bash : 컨테이너가 시작될 때 bash shell 실행

In [1]:
!docker run --gpus all -it --rm -v ./data:/dataset -v ./runs:/results stylegan2ada /bin/bash 
# vscode 터미널에서는 지원하지 않음
# 아래 코드와 같이 winpty 추가

the input device is not a TTY.  If you are using mintty, try prefixing the command with 'winpty'


In [2]:
!winpty docker run --gpus all -it --rm -v ./data:/dataset -v ./runs:/results stylegan2ada /bin/bash

'winpty'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.
